In [1]:
import sys
sys.path.insert(0, "..") # pseries_basis is here
from pseries_basis import * # Loading our package

%display latex
# n = PSBasis(QQ).n()

# Inverse Zeibelger Problem $\def\cc{\mathbb{C}}\def\nn{\mathbb{N}}\def\kk{\mathbb{K}}\def\qq{\mathbb{Q}}\def\zz{\mathbb{Z}}\def\basis{\mathfrak{B}}\def\ibasis{\mathfrak{I}}\def\pbasis{\mathfrak{P}}\def\bbasis{\mathfrak{C}}\def\hbasis{\mathfrak{H}}$

In this notebook we implement Petkovšek's algorithm `DefiniteSumsSolutions` in Sage using the package ``ore_algebra`` by M. Kauers and M. Mezzaroba to manipulate the differential and difference linear operators.

The main idea of this algorithm is translating recurrence equations that has no simple solution to a new difference equation which we may can compute a solution after assuming some hypergeometric terms in the original solution.

### Input and output

The algorithm receives a P-finite equation $L$ and some numbers $a_1,...,a_m \in \nn$ and some elements $b_1,...,b_m \in \kk$.

The algorithm returns another P-finite equation $L'$, such that $(h_k)_{k\geq 0}$ is a solution of $L'\cdot h = 0$ if and only if
$$L\cdot\left(\sum_{k\geq0}\prod_{i=1}^m\binom{a_in+b_i}{k}h_k\right) = 0$$

## 1. Compatible basis

The idea of this algorithm is to change the usual representation of a sequence stablishing a different basis. In this work we have two main perspectives: working with sequences or working with formal power series. They are easily related with a natural bijection:

$$(a_n)_n \leftrightarrow \sum_{n} a_n x^n.$$

In both cases, the "canonical" basis is the trivial $x^k = e_k(x) \equiv (e_{k,n})_n = (\delta_{n,k})_n$. However, there are other basis that, sometimes, help to write and solve a prolem from a new angle.

Let $\basis = \{P_k(n)\ :\ k\in\nn\}$ be a basis over $\kk$ of the ring of sequences $\kk^\nn$. Although Petkovšek is mainly interested in basis formed by polynomials, we are going to include here also other type of power series basis.

* $\basis$ is said to be a **polynomial basis** if $\deg(P_k) = k$.
* $\basis$ is said to be an **order basis** if (as power series) $ord_x(P_k) = k$.
* $\basis$ is **$(A,B)$-compatible** for a $\kk$-linear operator $L$ if for any $k \in \nn$
  $$L\cdot P_k(n) = \sum_{i = -A}^{B} \alpha_{i}(k)P_{k+i}(n),$$
  i.e., applying the linear operator $L$ to $P_k(n)$ expands the basis with a finite bound around $P_k(n)$.
  
Petkovšek is also interested in a particular type of polynomial basis that will prove to be useful for the following results. Here we take the chance to add the definition for orthogonal polynomials.

* $\basis$ is a **factorial basis** if for all $k \in \nn$, $P_k(n) | P_{k+1}(n)$. This means, there are $(a_k)\in (\kk^*)^\nn$ and $(b_k) \in \kk^\nn$ such that $P_{k+1}(n) = (a_kn + b_k)P_k(n)$.
* $\basis$ is a **orthogonal basis** if it is a *polynomial basis* such that for all $k \in \nn$ there are $(a_k)_k$, $(b_k)_k$ and $(c_k)_k$ in $\kk^\nn$ such that
  $$P_{k+1}(n) = (a_kn + b_k)P_k(n) - c_kP_{k-1}(n).$$ 

### Example of compatible operators

In this section we will see how the usual polynomial basis are compatible with some basic linear operators. We will follow the notation:

* $\pbasis$ will denote the _power basis_ where $P_k(n) = n^k$.
* $\pbasis_{a,b}$ will denote the _general power basis_ where $P_k(n) = (an + b)^k$.
* $\bbasis$ will denote the _binomial basis_ where $P_k(n) = \binom{n}{k}$.
* $\bbasis_{a,b}$ will denote the _general binomial basis_ where $P_k(n) = \binom{an+b}{k}$.
* $\hbasis$ will denote the basis of orthogonal _Hermite polynomial_ $P_k(n) = H_k(n)$

And for the operators we will use:

1. $D_x$ for standard derivation in $\kk[[x]]$ and $D_n$ the derivation $\partial_n(P_k(n))$.
2. $E$ for the _shift_ operator $n \rightarrow (n+1)$.
3. $Q$ for the _q-shift_ operator $n \rightarrow (qn)$.
4. $X$ for the _multiplication by $n$_ operator.

Then it is clear by definition that:
* Every polynomial factorial basis is $(0,1)$-compatible with $X$, since $P_k(n)(a_kn + b_k) = P_{k+1}(n)$, so:
  $$nP_k(n) = \frac{1}{a_k}\left(P_{k+1}(n) - b_kP_k(n)\right).$$
* Every orthogonal basis is $(-1,1)$-compatible with $X$ using the three terms recurrence:
  $$nP_k(n) = \frac{1}{a_k}\left(c_kP_{k-1}(n) - b_kP_k(n) + P_{k+1}(n)\right)$$
* $\pbasis_{a,b}$ is $(1,0)$-compatible with $D_n$:
  $$((an+b)^k)' = ak(an+b)^{k-1}.$$
* $\pbasis$ is $(0,0)$-compatible with $Q$:
  $$Q(n^k) = q^k(n^k).$$
* $\pbasis$ is not compatible with $E$: since $(n+1)^k = \sum_{i=0}^n\binom{n}{i}n^i$.
* $\bbasis$ is $(1,0)$-compatible with $E$:
  $$\binom{n+1}{k} = \binom{n}{k} + \binom{n}{k-1}.$$
* $\bbasis_{a,b}$ is $(a,0)$-compatible with $E$ (as it is shown in Proposition 6 of Petkovšek's paper):
  $$\binom{a(n+1)+b}{k} = \sum_{i=-a}^0\binom{a}{-i} \binom{an+b}{k+i}.$$
* $\hbasis$ is $(1,0)$-compatible with $D_n$ since:
  $$H_k'(n) = 2kH_{k-1}(n).$$

##### What is it implemented?

In the package `pseries_basis`, we have implemented this system of basis and compatibility. The following classes are available:
* `PSBasis`: a class that represent a "sequence of sequences". This class allows to add the compatibilities and compute the corresponding recurrences.
* `FactorialBasis`: a class specializing `PSBasis` that represents the factorial bases described in the paper. 
* `OrthogonalBasis`: a class specializing `PSBasis` that represents any _orthogonal basis_. It works similarly to `FactorialBasis`.

Moreover, several of the examples are implemented in different classes to simplify the input:
* `PowerTypeBasis`: allows to create $\pbasis_{a,b}$ by giving $a$ and $b$. The object `PowerBasis` is the instance for $a=1$ and $b=0$. It includes the compatibility with $D_n$.
* `BinomialTypeBasis`: allows to create $\bbasis_{a,b}$ by giving $a$ and $b$. The object `BinomialBasis` is the instance for $a=1$ and $b=0$. It includes automatically the compatibility with $E$.
* `FallingBasis`: represents basis $\basis = \{P_k(n)\}_n$ where the polynomial $P_k(n)$ can be written as:
$$P_k(n) = \prod_{i=0}^{k-1}(an+b-ic).$$
  These bases include the falling factorial ($a=1,b=0,c=1$) and raising factorial basis ($a=1,b=1,c=-1$). These basis are compatible with the shift $\tilde{E}: n \mapsto n + c/a$. This class includes automatically the compatibilities by $X$ and $\tilde{E}$. The objects `FallingFactorial` and `RaisingFactorial` are included in the code.
* `HermiteBasis`: represents $\hbasis$. It includes the compatibility with $D_n$.

In [2]:
show(PowerBasis)
PowerBasis.basic_compatibilities()

Basis of Sequences over Rational Field: (n^k)

['n', 'Id', 'Dn']

The method `compatibility(L)` receives the operator $L$ (either its name or the object in an `OreAlgebra` and returns the  compatibility description of the basis with such operator. A compatibility is built from the data $(A,B,t,\alpha_{i,j}(n))$, where:

* $A$ is the lower bound of the compatibility condition. See also the method `lower_bound` or simply `A`.
* $B$ is the upper bound of the compatibility condition. See also the method `upper_bound` or simply `B`.
* $t$ is the number of sections in which the compatibility is divided.
* $\alpha_{b,i}(m)$ are the coefficients of the compatibility such that, for all $k = mt + b$:

  $$L \cdot P_k(n) = \sum_{i=-A}^B \alpha_{b,i}(m)P_{k+i}(n).$$
  
The format for the $\alpha_{b,i}(m)$ is list of lists of sequences where $b$ and $i$ are the two indices of the lists and $m$ can be seen as the argument that defines a full sequence. The more structure on the sequence, the better information we can get from these methods.

In [3]:
PowerBasis.compatibility('Dn')

Compatibility condition (1, 0, 1) with following coefficient matrix:
[k 0]

In [4]:
P11 = PowerTypeBasis(1,1)
show(P11)
P11.basic_compatibilities()

Basis of Sequences over Rational Field: ((n + 1)^k)

['n', 'Id', 'Dn']

In [5]:
show(BinomialBasis)
print(all(BinomialBasis(i).generic() == binomial(var('n'),i) for i in range(20)))
BinomialBasis.basic_compatibilities()

Basis of Sequences over Rational Field: (binomial(n, k))

True


['n', 'E']

In [6]:
H = HermiteBasis();
show(H)
print(all(H.element(i).generic() == hermite(i,SR('n')) for i in range(20)))
H.basic_compatibilities()

Basis of Sequences over Rational Field: (hermite(k, n))

True


['n', 'Dn']

### Scalar equivalence

One basic operation for series of the power series is the scalar multiplication. Let $\basis$ be a basis of $\kk[[x]]$ with general term $P_n(x)$ and $(c_n)_{n\in \kk}$ be a sequence of constants such that $c_n \neq 0$. Then the sequence $(c_nP_n(x))_{n\in \kk}$ is also a basis of $\kk[[x]]$. Moreover:

**Lemma:** let $\basis = (P_n(x))_n$ be a basis of $\kk[[x]]$, $(c_n)_n \in \kk^\nn$ and $L$ a $(A,B)$-compabitle operator with $\basis$. Let $Q_n(x) = c_nP_n(x)$ and $\basis' = (Q_n(x))_n$. Then:
* If $\basis$ is factorial, then $\basis'$ is factorial.
* If $\basis$ is an order basis, then $\basis'$ is an order basis.
* $L$ is $(A,B)$-compatible with $\basis'$.

The first two points need no proof, since $c_n \in \kk \setminus \{0\}$, the order and degree are preserved. Moreover, if $P_n(x)$ are polynomials, multiplying by $c_n$ does not change the roots, hence if $\basis$ is factorial, so is $\basis'$.

Now, assume that the compatibility condition of $L$ with $\basis$ can be written as:
$$L(P_n(x)) = \sum_{i=-A}^B \alpha_{n,i}P_{n+i}(x),$$
then, using the fact that $L$ is $\kk$-linear:
$$L(c_nP_n(x)) = \sum_{i=-A}^B (\alpha_{n,i}\frac{c_n}{c_{n+i}} (c_{n+i}P_{n+i}(x)).$$

##### How is it implemented?

In order to obtain a suitable quotient $c_{n}/c_{n+i}$ for all $n$, we require that the sequence $(c_n)_n$ is hypergeometric. This means that these quotients are always rational functions in $i$ and $n$, and allows us to naively represent them into the system.

The class ``PSBasis`` defines a generic method ``scalar`` for scalar multiplication that first checks that the input is, indeed, hypergeometric. Depending if the sequence is a rational function or hypergeometric, this method will call either the method
``_scalar_basis`` or ``_scalar_hypergeometric``. These functions are specific for each class of the series implemented and return the structure for the new basis.

After that, the method ``scalar`` adds all the compatibilities that were computed already for the original basis.

We also allow the use of magic python syntax ``*`` with elements that can be casted into rational functions in $n$.

**_Remark_**: there a new standard class called `ScalarBasis` that uses as iput a normal Factorial basis and the sequence $(c_n)_n$ as structure to the whole basis. Sometimes we avoid the use of this extra layer (when we know the exact type for the scalar product). Otherwise, we use this class.

**_Remark 2_**: for non-Factorial basis, the class use to represent generic scalar products is the class `BruteBasis`. This class simply has a method `element` that receives the value of $n$ and returns the $n$-th element of the basis. This class **does not** guarantee that the method returns a basis, so its use is strongly discouraged.

In [7]:
monic_Binomial = BinomialBasis.scalar(factorial(var('k'))); show(monic_Binomial)
monic_Binomial.basic_compatibilities()

Basis of Sequences over Rational Field: (binomial(n, k)*factorial(k))

['n', 'E']

We can see how the Binomial basis and the Falling factorial basis are very related:

In [8]:
monic_Binomial == FallingFactorial

True

And we can see also that the compatibilities are equivalent for both:

In [9]:
all(monic_Binomial.compatibility(op).equiv(FallingFactorial.compatibility(op)) for op in monic_Binomial.basic_compatibilities())

True

## 2. From compatible operators to recurrence equations

In Proposition 4, Petkovšek proved that, givel a basis $\basis = \{P_k(n)\ :\ k \in \nn\}$ and a linear operator $L$ which is $(A,B)$-compatible, then it is equivalent, for $y(n) = \sum_{k\geq0}c_kP_k(n)$:
* $L\cdot y(n) = 0$.
* For all $k \geq 0$:
  $$\sum_{i=-B}^A \alpha_{-i}(k+i)c_{k+i} = 0,$$
  where $c_k = 0$ whenever $k < 0$.
  
This leads to a recurrence equation for the $c_k$:
$$(R_{\basis}L)\cdot (c_k) = \left(\sum_{i=-B}^A \alpha_{-i}(k+i)S_k^i\right)(c_n) = 0.$$
Hence, whenever the $\alpha_{i}(k)$ are rational functions in $k$, we arrive to a new P-finite recurrence equation.

Moreover, in Theorem 1, Petkovšek proved that the map from $L$ to $R_{\basis}L$ is a ring isomorphism, which alow to simple computations of the recurrence operators. Given $L_1, L_2$ compatible with $\basis$ and $c \in \kk$, then we have:
* $R_{\basis}(cL_1) = cR_{\basis}(L_1)$.
* $R_{\basis}(L_1+L_2) = R_{\basis}(L_1) + R_{\basis}(L_2)$.
* $R_{\basis}(L_1L_2) = R_{\basis}(L_1)R_{\basis}(L_2)$.

This means that, if we have $L \in \kk[O_1,\dots,O_t]$ where $O_i$ are compatible operators with $\basis$, then to compute $R_{\basis}(L)$ we only need to evaluate the polynomial that represents $L$:
    $$L = Q(O_1,\dots,O_t) \Rightarrow R_{\basis}(L) = Q(R_{\basis}(O_1),\dots,R_{\basis}(O_t).$$

#### What is it implemented?

In our package `pseries_basis` we have implemented this isomorphism with several methods:
* `recurrence(L)`: this method computes $R_{\basis}(L)$. This method requires that $L$ is expressed as a polynomial of operators that have been already set. 

The user can specified further compatibilities by giving the appropriate name for it and the corresponding value of $R_{\basis}(\cdot)$ or the values for $(A,B,t,\alpha_{b,i}(m))$ described previously.
* `set_compatibilty(L, R)`: set the compatibility of $L$ to $R$.

***REMARK***: the method ``set_compatibility`` only transforms recurrence to compatibilities assuming the value of $m$ is 1 (i.e., only 1 section is allowed). This is because, when we have multiple sections, the map from compatibilities to recurrence is not a isomorphism.

In [10]:
BinomialBasis.compatibility("n")

Compatibility condition (0, 1, 1) with following coefficient matrix:
[    k k + 1]

In [11]:
BinomialBasis.compatibility("E")

Compatibility condition (1, 0, 1) with following coefficient matrix:
[1 1]

##### Example 4 from Petkovšek's paper

* At example 4.1, Petkovšek considers the operator $L = E - c$, so $y(n) = \sum_{k \geq 0} y_k \binom{n}{k}$ satisfies $y(n+1) - c = 0$ if and only if $L' \cdot (y_k) = 0$ where $L'$ is the changed operator to the binomial basis:

In [12]:
## Creating the Binomial Basis with the parameter "c"
B2 = BinomialBasis.change_base(QQ['c'])
c = B2.base.0

In [13]:
B2.recurrence("E - c", output="ore_double") # has to get Sk - c + 1

Sk - c + 1

  In general, $L' = S_k - (c-1)$, which in the sequence level has the solution $y_n = y_0(c-1)^n$, obtaining:
  $$y(n) = \sum_{k \geq 0} \binom{n}{k}y_k(c-1)^k = y_0c^n$$
  
* At example 4.2, Petkovšek considers the operator $L = E^2 - 2E + 1$. Then, applying the substitution for a binomial basis we get:

In [14]:
BinomialBasis.recurrence("E^2 - 2*E + 1", output="ore_double") # has to get Sk^2 (currently wrong)

Sk^2

What does this result mean? Let $y(n) = \sum_{k\geq 0} y_k\binom{n}{k}$. Then we have that $y_{k+2} = 0$ for all $k$, so $y(n) = y_0 + y_1\binom{n}{1} = y_0 + y_1n$ and, then, all the power series solutions to the difference equation
$$y(n+2) - 2y(n+1) + y(n) = 0,$$
are the polynomials of degree 1.

* At example 4.3, Petkovšek now consider the operator $L = E^2 - E - 1$. Applying the algorithm we get:

In [15]:
BinomialBasis.recurrence("E^2 - E - 1", output="ore_double") # has to get Sk^2 + Sk - 1

Sk^2 + Sk - 1

In this case we are considering a Fibonacci type of function, some $y(n)$ such that $y(n+2) = y(n+1) + y(n)$. If we expand it using the binomial basis, we end up that the sequence of coefficients satisfies the recurrence
$$y_{k+2} = y_k - y_{k+1},$$
which is a variance of the Fibonacci sequence. In fact, we have:
$$y_k = (-1)^k(C_1F_k + C_2F_{k+1}),$$
where $F_k$ **is** the Fibonacci sequence $(0,1,1,2,\dots)$. Let choose $C_1$ and $C_2$ such that $y(0) = 0$ and $y(1) = 1$. Then we have that $0 = y_0 = C_2$ and $1 = y_0 + y_1 = -(C_1 + C_2)$, so $C_1 = -1$ and $C_2 = 0$. Then $y_k = (-1)^kF_k$ and we have:
$$y(n) = \sum_{k \geq 0} (-1)^kF_k,$$
so evaluating the function on an integer $m$ we obtain:
$$F_m = \sum_{k = 0}^m (-1)^kF_k.$$

* At example 4.4, Petkovšek takes the P-finite first order recurrence $L = E - (n+1)$. After applying the substitution we get:

In [16]:
BinomialBasis.recurrence("E - (n+1)", output="ore_double") # has to get Sk - kSki - k

Sk + (-k)*Ski + (-k)

In this case we have a recurrence with the inverse shift, which is equivalent to the recurrence $S_k^2 - k S_k - k$. This recurrence equation has as solution $y_k = k!\left(C_1 + C_2\sum_{l=1}^k(-1)^l/l!\right)$.

However in this case for $k = 0$, the original equation (with the right shift operator) provides the equality $y_1 = 0$, so we obtain $C_1 = C_2 = C$ obtaining finally
$$y(n) = C\sum_{k \geq 0} \binom{n}{k}k!\left(1 + \sum_{l=1}^k \frac{(-1)^l}{l!}\right).$$

Now, in this case, the original recurrence gives that $y(n+1) = (n+1)y(n)$ (which is the functional equation for the factorial: $y(n) = n!y(0)$). Hence, we find the following identity setting $C=1$:
$$m! = \sum_{k = 0}^m \binom{m}{k}k!\left(1 \sum_{l=1}^k\frac{(-1)^l}{l!}\right).$$

* Finally in example 4.5, Petkovšek studies the recurrence $L = E^3 - (n^2+6n+10)E^2 + (n+2)(2n+5)E-(n+1)(n+2)$. This recurrence is not so simple anymore.

In [17]:
BinomialBasis.recurrence("E^3 - (n^2+6*n+10)*E^2 + (n+2)*(2*n+5)*E-(n+1)*(n+2)", output="ore_double") 
# has to get Sk^3 + (-k^2 - 6k - 7)Sk^2 + (-2k^2 -8k -7)Sk - (k^2 + 2k +1)

Sk^3 + (-k^2 - 6*k - 7)*Sk^2 + (-2*k^2 - 8*k - 7)*Sk + (-k^2 - 2*k - 1)

## 3. The equivalence of Proposition 2 **TODO**: review this when increase polynomials are updated

In Proposition 2, Petkovšek provides an equivalent condition, in the case of *polynomial factorial basis*, for an operator $L$ to be $(A,B)$-compatible. Namely, $L$ is $(A,B)$ compatible if and only if:
* **C.1.** For all $n \geq 0$, $\deg(L\cdot P_n) \leq n + B$.
* **C.2.** For all $n \geq A$, $P_{n-A} | L\cdot P_n$.

This proposition will be heavily used to proved compatibility for some operators in the _product basis_ defined below. Hence, a constructive proof is heavily desired. 

### 3.1 Defining the goals for equivalence

A first step to make the result constructive is define exactly what we need to compute. For the compatibility condition is pretty clear: we need to compute the values $A$, $B$, $m$ and $\alpha_{i,j}(n)$ for arbitrary (symbolic) $n$, $0 \leq i \leq m-1$ and $-A \leq j \leq B$. 

For the conditions **C.1** and **C.2**, we need to compute the sections $m$, the bound $B$, the bound $A$ and coefficients $c_{i,j}(n)$ for arbitrary $n$, $0 \leq i \leq m-1$ and $0 \leq j \leq A+B$ such that, for all $n = km + r$:

$$\frac{L\cdot P_{n}}{P_{n-A}} = \sum_{j=0}^{A+B}c_{i,j}(k)x^j.$$

### 3.2 Increasing basis

The main tool for performing this computation is what we will call the _increasing basis_. Using the fact that we are working with *polynomial factorial basis*, we have that for any $m > n$, $\deg(P_m/P_n) = m-n$. 
* We define the $k$*-increasing basis* for $\basis$ as $\ibasis_k(\basis) = \left\{I_{k,n}\right\}_{n\geq 0}$ where
  $$I_{k,n} = \frac{P_{k+n}}{P_k}.$$
* $\ibasis_k$ **is a polynomial factorial basis for any** $k \in \nn$.

  It is clear, as we have already mention, that $\deg(I_{k,n}) = n$. And, as $\basis$ is factorial, we also have that:
  $$\frac{I_{k,n+1}}{I_{k,n}} = \frac{P_{k+n+1}P_k}{P_{k+n}P_k} = \frac{P_{k+n+1}}{P_{k+n}} = a_{n+k+1}x + b_{n+k+1},$$
  finishing the proof. $\square$

### 3.3 Looking to the equivalence

The compatibility condition provides the following identity:
$$L(P_n) = \sum_{i=-A}^B \alpha_{n,i} P_{n+i},$$
so if we divide this identity by $P_{n-A}$, we obtain:
$$\sum_{j=0}^{A+B} c_{n,j}x^j = \frac{L(P_n)}{P_{n-A}} = \sum_{i=-A}^B \alpha_{n,i}I_{n-A,i+A} = \sum_{j=0}^{A+B} \alpha_{n,j-A}I_{n-A, j}.$$

This implies that the computations for changing from teh compatibility to the conditions **C.1** and **C.2** are simply a change of coordinates between the canonical basis $1,x,x^2,\ldots$ and the increasing basis at an arbitrary point $n$. The computations needs to be done carefully to carry over the details of the sections, but otherwise, these computations are straightforward.

#### What is it implemented?

The class `FactorialBasis` provides several methods to compute the equivalence of Proposition 2.

* `increasing_polynomial(k,n)`: returns the polynomial $I_{k,n}$ for the current basis. The value for $n$ must be a fixed positive integer and $k$ may take a symbolic value. This method will take care of getting the increasing basis splitted into sections in case the basis is splitted as well.
* `increasing_basis(k)`: if possible, returns the object representing the $k$-increasing basis.
* `matrix_ItP(k,m)`: returns the matrix $M_{I\rightarrow P}$ of size $m$ for the $k$-increasing basis that changes coordinates from the increasing basis to the canonical basis.
* `matrix_PtI(k,m)`: returns the matrix $M_{P\rightarrow I}$ of size $m$ for the $k$-increasing basis that changes coordinates from the canonical basis to the increasing basis.
* `equiv_DtC(compatibility)`: assuming that the input is a tuple with $(A,B,m,\alpha_{i,j}(n))$ representing the compatibility of an operator $L$ with such that for all $n = km+r$ or the operator itself, we have that
  $$L(P_{n}) = \sum_{i=-A}^{B} \alpha_{r,i}(k)P_{n+i}.$$
  This method computes the coefficients $c_{i,j}(n)$ for the equivalent condition **C.2** (i.e., the coefficients of the polynomials $L(P_{n})/P_{n-A}$):
  $$\frac{L(P_n(x))}{P_{n-A}(x)} = \sum_{i = 0}^{A+B+1} c_{r,i}(k)x^k.$$
* `equiv_CtD(division)`: providing a `DivisionCondition`, we compute the equivalent compatibility condition (splitted in the same number of sections as deteced in the division). The output, then, will be a compatibility condition, where:
  $$L(P_{n}(x)) = \sum_{i=-A}^B \alpha_{r,i}(k)P_{n+i}(x).$$
* `compatible_division(L)`: this method computes generically the division $L\cdot P_n/P_{n-m}$ for some $m \geq A$. If any subclass requires further information for computing this division, we implement that particular version explicitly. The output is a tuple $(A, m, D_{i,j}(n))$ where $A$ is, as usual, the lower bound of the compatibility, $m$, the number of sections and $D_{i,j}(n)$ a function such that:
  $$\frac{L\cdot P_{km+r}}{P_{km+r-s}} = D_{r,s}(k) \in \mathbb{K}[x].$$

In [18]:
## Checking how the increasing polynomials are the divisions between elements of the basis
all(
    all(
        all(
            basis[i+j]
            == 
            basis[i]*basis.increasing_basis(i)[j] 
            for j in range(20)) 
        for i in range(4,20))
    for basis in [PowerBasis, P11, BinomialBasis]
)

True

In [19]:
## Checking how we get the coefficients c_{n,i}
BinomialBasis.equiv_CtD("E - (n+1)")

Divisibility condition (A=1, t=1) with following coefficient matrix:
[        1 (k - 1)/k      -1/k]

## 4. Recurrence in sections

In Proposition 9, Petkovšek studies how the fact that an operator is compatible with a basis makes that all the sections sequences in that basis satisfies different recurrence relations. The prove of the result is quite technical but the only thing that is important is that, for $m > 0$, if an operator is compatible in the form
$$L(P_{km+j}) = \sum_{i=-A}^B \alpha_{k,j,i} P_{km+j+i},$$
then we can define the recurrence operators for $r,j \in \{0,1,\dots,m-1\}$:
$$L_{r,j} = \sum_{\begin{array}{c}-A\leq i \leq B\\i+j \equiv r (m)\end{array}} \alpha_{k+\frac{r-i-j}{m},j,i} S_n^{\frac{r-i-j}{m}}.$$

Then with this operators we have that, for $y(x) = \sum_{k\geq 0} \sum_{j=0}^{m-1} c_{km+j}P_{km+j}$, $L(y) = 0$ if and only if for all $r \in \{0,\dots,m-1\}$ we have:
$$\sum_{j=0}^{m-1} L_{r,j}(c_{km+j})_k = 0.$$

### 4.1 Matrix operator

If we form now $R_{\basis}^m(L) = \left(L_{r,j}\right)_{r,j=0}^{m-1}$, as an extension of the original map $R$ that we saw in previous sections, then it still have the same properties: it is a ring homomorphism.

For any $m > 0$, $L_1, L_2$ compatiblo operators with $\basis$ and $c \in \kk$:
* $R_{\basis}^m(cL_1) = cR_{\basis}^m(L_1)$.
* $R_{\basis}^m(L_1 + L_2) = R_{\basis}^m(L_1) + R_{\basis}^m(L_2)$.
* $R_{\basis}^m(L_1L_2) = R_{\basis}^m(L_1)R_{\basis}^m(L_2)$.

#### What is it implemented?

From the compatibility condition that we obtain with method `compatibility`, i.e., a tuple of the form $(A,B,m,\alpha_{i,j}(n))$, we can always extend this compatibility condition to any number of sections that is divisible by $m$.

Let $p = lm$ for some $l \in \mathbb{N}$. Consider $r = r_0m + r_1$ with $r \in \{0,\ldots,p-1\}$ and $r_1\in \{0,\ldots,m-1\}$. Then we can easily write:

$$L\cdot P_{kp + r} = L\cdot P_{(kl + r_0)m + r_1}= \sum_{j=-A}^B \alpha_{r_1,j}(kl+r_0) P_{kp+r+j}.$$

Hence, we have that the new compatibility coefficients are:

$$\tilde{\alpha}_{r,j}(k) = \alpha_{r\%m,j}(kp/m + r//m).$$ 

This extension is implemented in the method ``compatibility_sections(L, p)`` and returns, if possible, the compatibility tuple $(A,B,p,\tilde{\alpha}_{i,j}(n))$.

The method ``recurrence`` (described above) accepts an optional parameter ``sections`` that fix the number of sections for getting the recurrence matrix. If given, we extend the compatibility to the given number of sections and then apply the transformation to a matrix of recurrences described before.

In [20]:
BinomialBasis.compatibility("n").in_sections(3)

Compatibility condition (0, 1, 3) with following coefficient matrix:
[    3*k 3*k + 1]
[3*k + 1 3*k + 2]
[3*k + 2 3*k + 3]

In [21]:
BinomialBasis.recurrence('n', output="ore_double")

k*Ski + k

In [22]:
BinomialBasis.recurrence('n', 3, output="ore_double")

[    (3*k)         0 (3*k)*Ski]
[(3*k + 1) (3*k + 1)         0]
[        0 (3*k + 2) (3*k + 2)]

## 5. Product basis

Petkovšek works with a type of basis that is composed with products of simpler basis. Namely with binomial basis. Let $\basis_1,\dots,\basis_m$ be basis of $\kk[[x]]$, we then define the new set:
$$Q_{km+r} = \prod_{i=1}^r P_{k+1}^{(i)} \prod_{i=r+1}^mP_k^{(i)}.$$

It is shown in Theorem 2 that $\basis = \prod \basis_i$ is a factorial basis when all $\basis_i$ are factorial. This provides that $X$ is $(0,1)$-compatible with $\basis$. Moreover, he shown that if $L$ is a ring homomorphism over $\kk[x]$, then $L$ is compatible with $\basis$. We can show also that if $L$ is a derivation over $\kk[[x]]$, then $L$ is also compatible with $\basis$.

This is a clear case where the compatibilities and other operations over the Product basis must be splitted into sections. In fact, this can be done in a more generic case (see below the section describing the Shuffled Basis). We know show a simple case where all the compatibilities of the factors have exactly one section (just as an illutration on how the reasoning works).

### 5.1 Extending compatibility of $X$

Extending the compatibility of $X$ is pretty simple since the multiplication of polynomials is commutative. Hence, if we pick $k \in \nn$ and $r \in \{0,\dots,m-1\}$, then we have:
$$xQ_{km+r} = \left(\prod_{i=1}^{r}P_{k+1}^{(i)}\right)(xP_{k}^{(r+1)})\left(\prod_{i=r+2}^mP_{k}^{(i)}\right),$$
then, using the fact that $xP_k^{(r+1)} = \alpha_{0,0}^{(r+1)}(k)P_k^{(r+1)} + \alpha_{0,1}^{(r+1)}(k)P_{k+1}^{(r+1)}$, we have that
$$xQ_{km+r} = \alpha_{0,0}^{(r+1)}(k)Q_{km+r} + \alpha_{0,1}^{(r+1)}(k)Q_{km+r+1},$$
so we can conclude that:
$$\tilde{\alpha}_{r,j}(k) = \alpha_{0,i}^{(r+1)}(k).$$

### 5.2 Extending compatibility of endomorphisms

Let $L$ be an endomorphism of $K[x]$, i.e., $L(PQ) = L(P)L(Q)$ for any pair of polynomials. Then we can follow the prove of compatibility on the product basis:

* Proving **C.1**: given $Q_n$, we can expand it in terms of the $j$th factor basis:
  $$Q_n = \sum_{i=0}^nq_{n,i}P_n^{(j)},$$
  and applying $L$ to this expression we get
  $$L(Q_n) = \sum_{i=0}^n q_{n,i}\sum_{k=-A_j}^{B_j} \alpha_{0,k}^{(j)}(i) P_{i+k},$$j
  and a simple scan of that last expression shows that $\deg(L(Q_n)) \leq n + B_j$. As this hold for all $j = 1,...,m$, then we can conclude $\deg(L(Q_n)) \leq n+\min(B_j)$. So $B=\min(B_j : 1 \leq j \leq m)$.
  
  ***REMARK:** This proof only uses that $L$ is compatible with all the factors of $\basis$.*
  
* Proving **C.2**: if we apply directly $L$ to the product expression of $Q_n$ for $n = km+r$, we obtain:
  $$L(Q_n) = \left(\prod_{i=1}^r \sum_{l=-A_i}^{B_i} \alpha_{0,l}^{(i)}(k+1)P_{k+1+l}^{(i)}\right) \left(\prod_{i=r+1}^{m} \sum_{l = A_i}^{B_l} \alpha_{0,l}^{(i)}(k)P_{k+l}^{(i)}\right).$$
  
  If we expand this expression and see the summands, we can see they involve some product of elements from the factor basis. In fact, the minimal index we see in each case is $P_{k-A_i}^{(i)}$. Then, using the fact that all the basis are factorial, we have that for $A = \max(A_j : 1 \leq j \leq m)$, we have:
  $$Q_{n-mA} = \prod_{i=1}^r P_{k-A+1}^{(i)} \prod_{i=r+1}^mP_{k-A}^{(i)} | L(Q_n).$$
  
This two points prove that $L$ is $(mA, B)$-compatible with the product basis.

##### How is it implemented?

In order to get a constructive approach for this prove, we need to recall what we discussed in Section 3: we need to construct the division $L(Q_n)/Q_{n-mA}$. As usual, and for later use, we will consider $n = km+r$ and each of the $r$ cases will be taken separately.

If we analyze the quotient we want to study, we realize that:
$$\frac{L \cdot Q_{km+r}}{Q_{km+r-mA}} = \prod_{i=1}^r\left(\frac{L \cdot P_{k+1}^{(i)}}{P_{k+1-A}^{(i)}}\right)\prod_{i=r+1}^m\left(\frac{L \cdot P_{k}^{(i)}}{P_{k-A}}\right).$$
Each of these quotients can be coputed explicitly using the method ``compatible_division`` for each of the factorial basis, 
obtaining then that:
$$\frac{L \cdot Q_{km+r}}{Q_{km+r-mA}} = \prod_{i=1}^r D_{0,A}^{(i)}(k+1)\prod_{i=r+1}^{m} D_{0,A}^{(i)}(k) \in \mathbb{K}[x].$$

Moreover, we can also compute the corresponding *increasing basis* from a fixed $n = km+r$. Hence we can apply the corresponding change of coordinates to get the coefficients $\alpha_{r,j}(k)$. This provides directly the compatibility tuple desired.

* When creating a Product Basis, the user can specify the names of all the endomorphism he wants to compute the compatibility matrix. See the documentation of `ProductBasis?` for further information.

### 5.3 Extending compatibility of derivations

Let $L$ be a derivation of $K[x]$, i.e., $L(PQ) = L(P)Q + PL(Q)$ for any pair of polynomials. Then we can follow the prove of compatibility on the product basis:

* Proving **C.1**: exactly the same proof as for endomorhisms (see the remark above). Then $\deg(L(Q_n)) \leq n + B$ where $B = \min(B_j : 1 \leq j \leq m)$.
  
* Proving **C.2**: if we apply directly $L$ to the product expression of $Q_n$ for $n = km+r$, we obtain a summation of several terms:
  $$L(Q_n) = \sum_{i=1}^r L(P_{k+1}^{(i)})\frac{Q_n}{P_{k+1}^{(i)}} \sum_{i=r+1}^m L(P_{k}^{(i)})\frac{Q_n}{P_{k}^{(i)}}.$$
  
  Take $A = \max(A_j : 1 \leq j \leq m)$. We will show now that $Q_{n-Am}$ divides $L(Q_n)$ analyzing the divisibility of each of the summands. Let $i \leq r$:
  $$\frac{L(P_{k+1}^{(i)})Q_n}{Q_{n-mA}P_{k+1}^{(i)}}$$
  is a polynomial because the $i$th factor of $Q_{n-mA}$ (which is $P_{k+1-A}$) divides $L(P_{k+1}^{(i)})$ by th compatibility of $L$ with $\basis_i$. The $i$th factor of $Q_n$ is cancelled by $P_{k+1}^{(i)}$ and all the other factors are cancelled by $Q_{n-mA}$ since all the other basis are factorial.
  
  We can argue similarly for any $i \in \{r+1,\ldots m\}$. Hence $Q_{n-mA}$ divides $L(Q_n)$ for all $n \in \mathbb{N}$.
  
This two points prove that $L$ is $(mA, B)$-compatible with the product basis.

##### How is it implemented?

In order to get a constructive approach for this prove, we need to recall what we discussed in Section 3: we need to construct the division $L(Q_n)/Q_{n-mA}$. As usual, and for later use, we will consider $n = km+r$ and each of the $r$ cases will be taken separately.

Following the proof we have written above, the quotient of $L(Q_n)/Q_{n-mA}$ has a very special shape. More precisely:

$$\frac{L \cdot Q_{km+r}}{Q_{(k-A)m + r}} = 
\sum_{i=1}^r \left(D_{0,A}^{(i)}(k+1)\prod_{j\neq i, j = 1}^{r} I_{k+1-A,A}^{(j)} \prod_{j = r+1}^m I_{k-A,A}^{(j)}\right) + 
\sum_{i=r+1}^m \left(D_{0,A}^{(i)}(k)\prod_{j = 1}^{r} I_{k+1-A,A}^{(j)} \prod_{j\neq i, j = r+1}^m I_{k-A,A}^{(j)}\right)$$

where $D_{i,j}(k)$ can be computed with the method ``compatible_division`` for each basis and the $I_{i,j}$ can be computed with the method ``increasing_polynomial`` for each basis. This yields a polynomial that can be transformed into a compatibility condition using the ideas of method ``equiv_CtD``.

* When creating a Product Basis, the user can specify the names of all the derivations he wants to compute the compatibility matrix. See the documentation of `ProductBasis?` for further information.

##### Example 6

In this example, Petkovšek studied several product of binomial basis and their compatibilities. Here we can see we can extract the same information automatically using our package:

In [25]:
example_6a = ProductBasis([BinomialTypeBasis(2,0,QQ), BinomialTypeBasis(3,0,QQ)])
example_6a.compatibility('E')

Compatibility condition (6, 0, 2) with following coefficient matrix:
[                                       0                    (4/9*k - 2/3)/(k - 1) (211/36*k^2 - 187/18*k + 10/3)/(k^2 - k)                      (131/12*k - 16/3)/k                         (21/2*k - 9/2)/k                                        6                                        1]
[           (-4/27*k^2 + 2/9*k)/(k^2 - 1)     (8/27*k^2 - 28/27*k + 8/9)/(k^2 - 1)       (20/9*k^2 - 4/3*k - 2/9)/(k^2 + k)   (131/18*k^2 - 1/3*k - 17/18)/(k^2 + k)                   (17/2*k + 7/2)/(k + 1)                        (4*k + 2)/(k + 1)                                        1]

In [26]:
example_6b = ProductBasis([BinomialTypeBasis(2,-1,QQ), BinomialTypeBasis(3,4,QQ)])
example_6b.compatibility('E')

Compatibility condition (6, 0, 2) with following coefficient matrix:
[                                                    0                  (4/9*k^2 - 50/9*k + 154/9)/(k^2 - k)              (211/36*k^2 + 55/6*k + 791/36)/(k^2 - k)                                  (131/12*k - 97/12)/k                                     (21/2*k + 13/2)/k                                                     6                                                     1]
[   (-4/27*k^3 + 2/9*k^2 + 44/3*k - 1694/27)/(k^3 - k) (8/27*k^3 - 116/27*k^2 + 508/27*k - 616/27)/(k^3 - k)                (20/9*k^2 - 188/9*k + 416/9)/(k^2 + k)               (131/18*k^2 - 13/6*k + 107/9)/(k^2 + k)                               (17/2*k - 15/2)/(k + 1)                                     (4*k + 2)/(k + 1)                                                     1]

In [27]:
example_6c = ProductBasis([BinomialTypeBasis(4,0,QQ), BinomialTypeBasis(4,0,QQ)])
example_6c.compatibility('E')

Compatibility condition (8, 0, 2) with following coefficient matrix:
[                                                  1    (8*k^3 - 36*k^2 + 44*k - 12)/(k^3 - 3*k^2 + 2*k) (28*k^3 - 112*k^2 + 128*k - 36)/(k^3 - 3*k^2 + 2*k)                     (56*k^2 - 112*k + 40)/(k^2 - k)                     (70*k^2 - 126*k + 44)/(k^2 - k)                                       (56*k - 28)/k                                       (28*k - 12)/k                                                   8                                                   1]
[                                    (k - 3)/(k + 1)                                   (8*k - 8)/(k + 1)              (28*k^3 - 56*k^2 + 16*k + 4)/(k^3 - k)              (56*k^3 - 84*k^2 + 12*k + 8)/(k^3 - k)                      (70*k^2 - 14*k - 12)/(k^2 + k)                              (56*k^2 - 8)/(k^2 + k)                                 (28*k + 12)/(k + 1)                                   (8*k + 4)/(k + 1)                                                   1]

## 6. The method `DefiniteSumSolutions`

The method `DefiniteSumSolutions` presented in Petkovšek's paper can be now easily implemented using all the tools described above. This method i implemented in the subpackage ``pseries_basis.misc.misc``.

In [28]:
print(DefiniteSumSolutions.__doc__)


        Petkovšek's algorithm for transforming operators into recurrence equations.
        
        This method is the complete execution for the algorithm **DefiniteSumSolutions** described in
        :doi:`10.1016/j.jsc.2022.11.002`. This methods takes an operator `L` and convert the problem
        of being solution `L \cdot y(n) = 0` to a recurrence equation assuming some hypergeometric
        terms in the expansion.
        
        The operator must be a difference operator of `\mathbb{Q}[x]<E>` where `E: n \mapsto n+1`.
        
        This function does not check the nature of the generator, so using this algorithm with different 
        types of operators may lead to some inconsistent results.
        
        INPUT:

        * ``operator``: difference operator to be transformed.
        * ``input``: the coefficients of the binomial coefficients we assume appear in the expansion
          of the solutions. This input can be given with the following formats:
          - ``

##### Example 7

In this example Petkovšek applies the method ``DefiniteSumSolutions`` to several operators obtaining diverse resutls:

In [2]:
example_7_basis = ProductBasis([BinomialBasis, BinomialBasis]);
example_7_basis

Shuffled Basis (0, 1) of the basis:
	- Basis of Sequences over Rational Field: (binomial(n, k))
	- Basis of Sequences over Rational Field: (binomial(n, k))

In [3]:
example_7_basis.recurrence('n')

[      k   k*Ski]
[(k + 1)       k]

In [4]:
example_7_basis.compatibility('n')

Compatibility condition (0, 1, 2) with following coefficient matrix:
[    k k + 1]
[    k k + 1]

In [5]:
example_7_basis.recurrence('E')

[                Sk + 1      (2*k + 1)/(k + 1)]
[                  2*Sk (k + 1)/(k + 2)*Sk + 1]

###### Example 7.1

Here, Petkovšek considered the linear operator $L = (n+1)E - 2(2n+1)$ and applies its algorithm to get a recurrence equation $L'$ such that
$$L'(c_k) = 0\quad \Rightarrow\quad L\left(\sum_{k \geq 0} c_k\binom{n}{k}^2\right) = 0.$$

In [6]:
OE, (n, E) = get_recurrence_algebra("n")

In [7]:
example_7_1 = (n+1)*E - 2*(2*n+1);
DefiniteSumSolutions(example_7_1, 1,1,0,0)

Sk^2 - Sk

Then we can conclude that the only solution of that form has $c_k = 1$, showing that
$$\left((n+1)E - 2(2x+1)\right)\cdot\left(\sum_{k\geq 0} \binom{n}{k}^2\right) = 0,$$
which makes complete sense, since
$$\sum_{k \geq 0} \binom{n}{k}^2 = \binom{2n}{n}$$
is annihilated by $L$.

###### Example 7.2

In the last example of the paper, Petkovšek studies the following difference operator:
    $$L = 4(2n+3)^2(4n+3)E^2 - 2(4n+5)(20n^2+50n+27)E + 9(4n+7)(n+1)^2.$$
    
Again, we are looking for a new recurrence equation $L'$ such that $L'(c_k) = 0$ implies that $L\left(\sum_{k\geq 0} c_k \binom{n}{k}^2\right) = 0$.

In [8]:
example_7_2 = 4*(2*n+3)^2*(4*n+3)*E^2 - 2*(4*n+5)*(20*n^2+50*n+27)*E + 9*(4*n+7)*(n+1)^2;
DefiniteSumSolutions(example_7_2, 1,1,0,0)

(k + 3/2)*Sk^2 + (-1/4*k - 1/2)*Sk

Or, equivalently as a monic operator:
$$L' = S_n - \frac{n+1}{2(2n+1)}.$$

Using other tools we can check that $h_k = \frac{1}{\binom{2k}{k}}$ is annihilated by $L'$, so we can conclude that the solution $(y_n)_{n\geq 0} = \left(\sum_{k\geq 0}c_k\binom{n}{k}\right)_{n\geq 0}$ is the sequence
$$y_n = \sum_{k \geq 0} \frac{\binom{x}{n}^2}{\binom{2k}{k}}.$$

## 7. Shuffled basis

We are going to condier now another type of generalization of the Product basis. Remember that for a Product basis, we started with a set of basis $\basis_1,\ldots,\basis_m$ and we _interlaced_ them in an appropriate way, namely:
    $$P_{qm+r}(x) = \prod_{i=1}^{r}P_{i,q+1}(x) \prod_{i=r+1}^{m}P_{i,q}(x).$$
    
If we consider polynomial basis and we look into the root sequence, we are interlecing the $m$ sequences of roots for each of the factors of our basis. But we could create a new sequence in a very different way. For example, if we have two sequences, we could create the following root sequence:
    $$[\rho_{1,1},\rho_{1,2},\rho_{2,1},\rho_{1,3},\rho_{1,4},\rho_{2,2},\ldots],$$
i.e., we take two roots from the first basis and one root from the second basis. We could even mix them even more. In this section we are going to study this type of basis, that will allow us to build more examples automatically.

**Definition:** Let $\basis_0,\ldots,\basis_{F-1}$ be factorial basis and $\mathbf{c} = (\sigma_0,\ldots,\sigma_{m-1}) \in \{0,\ldots, F-1\}^m$. We define the $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$ with the polynomials $Q_n(x)$ defined as follows (taking $n = km + r$):

$$Q_n(x) = \prod_{i=0}^{F-1}P_{e_i(n)}^{(i)}(x),$$

where $e_i(n) = kS_i + s_i(r)$ where:
* $s_i(r) = \#\{j \in \{0,\ldots,r-1\ :\ \sigma_r = i\}$,
* $S_i = s_i(m)$.

This sieved basis do precisely the interlacing of root sequences that we were describing before. Namely, at step $n = km+r$ we add the next root of the basis $\basis_{\sigma_r}$. This generalizes the concept of **Product basis** since, when $m = F$ and $\mathbf{c} = (0,1,\ldots,F-1)$, we have that the $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$ is exactly the Product basis of the basis $\basis_0,\ldots,\basis_{F-1}$.

**Lemma:** Let $\basis_0,\ldots,\basis_{F-1}$ be factorial basis and $\mathbf{c} = (\sigma_0,\ldots,\sigma_{m-1}) \in \{0,\ldots, F-1\}^m$. The $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$ is a factorial basis.

*Proof:* let $n = km + r \geq 1$ and $f = \sigma_r$. Then, by definition, we have:
$$\frac{Q_{n}(x)}{Q_{n-1}(x)} = \frac{P_{e_f(n)}^{(f)}(x)}{P_{e_f(n)-1}^{(f)}(x)} \in \mathbb{K}[x]_{\leq 1}.$$

The last we need to proof that $\basis$ is a factorial basis is that the degree of the $n$-th element is exaclty $n$. This is equivalent to proof that $\sum_{i=0}^{F-1} e_i(n) = n$ for all $n$. But this is clear since:
* $\sum_{i = 0}^{F-1} S_i = m$ by definition
* $\sum_{i = 0}^{F-1} s_i(r) = r$ by definition.

Hence, putting these two pieces together we obtain for $n = km+r$:
$$\sum_{i=0}^{F-1} e_i(n) = \sum_{i=0}^{F-1}\left(kS_i + s_i(r) \right) = km+r = n.$$

**Fixed notation**: for the rest of this section, we will fix the following notation:
* $\basis_i = \{P_n^{(i)}(x)\}_n$ for $i = 0,\ldots,F-1$ are factorial basis.
* $\mathbf{c} = (\sigma_0,\ldots,\sigma_{m-1}) \in \{0,\ldots,F-1\}^m$
* $\basis$ is the $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$.

### Extending the compatibility of multiplication by $X$

Since the basis $\basis$ is again a factorial basis, it is cler that it is $(0,1)$-compatible with the multiplication by $X$. Here we present how to compute this compatibility from the compatibility conditions of each of the factors $\basis_i$.

Let $\basis_i$ be (0,1)-compatible with $X$ in $t_i$ sections. Then we have rational functions $\alpha_{r,j}^{(i)}(k) \in \mathbb{K}(k)$ such that, for all $n = kt_i + r$:

$$xP_{n}^{(i)}(x) = \alpha_{r,0}^{(i)}(k)P_{n}^{(i)}(x) + \alpha_{r,1}^{(i)}(k)P_{n+1}^{(i)}(x).$$

Let $t = \text{lcm}(t_0, \ldots, t_{F-1})$ and $a_i = t/t_i$. Then $X$ is $(0,1)$-compatible with $\basis$ in $mt$ sections. Let $n = kmt + r$ and write $r = r_0m + r_1$. Then we have that:

$$xQ_n(x) = xQ_{(kt+r_0)m+r_1}(x) = x\prod_{i=0}^{F-1}P_{e_i(n)}^{(i)}(x) = 
\left(\prod_{i=0,\ i\neq\sigma_{r_1}}^{F-1}P_{e_i(n)}^{(i)}(x)\right)\left(xP_{e_{\sigma_{r_1}}(n)}^{(\sigma_{r_1})}(x)\right).$$

Using the facts that:
* $e_{\sigma_{r_1}}(n) = S_{\sigma_{r_1}}(kt + r_0) + s_{\sigma_{r_1}}(r_1)$,
* $e_{\sigma_{r_1}}(n+1) = e_{\sigma_{r_1}}(n) + 1$ and,
* $S_{\sigma_{r_1}}r_0 + s_{\sigma_{r_1}}(r_1) = r_2t_{\sigma_{r-1}}+ r3$,

then we can write:
$$xQ_n(x) = \left(\prod_{i=0,\ i\neq\sigma_{r_1}}^{F-1}P_{e_i(n)}^{(i)}(x)\right)\left(xP_{t_{\sigma_{r_1}}(a_{\sigma_{r_1}}k+r_2) + r_3}^{(\sigma_{r_1})}(x)\right) = 
\alpha_{r_3,0}^{(\sigma_{r_1})}(a_{\sigma_{r_1}}k+r_2)Q_n(x) + \alpha_{r_3,1}^{(\sigma_{r_1})}(a_{\sigma_{r-1}}k+r_2)Q_{n+1}(x)$$

and we have computed the compatibilty conditions for $\basis$ taking:

$$\alpha_{r,j}(n) = \alpha_{r_3,j}^{(\sigma_{r_1})}\left(a_{\sigma_{r-1}}n+r_2\right) \in \mathbb{K}(n)$$

### Extending the compatibility of endomorphisms

Let $L$ be an endomorphism, i.e., $L(PQ) = L(P)L(Q)$. Then we can compute the compatibility condition in several sections for $\basis$ if $L$ is compatible with all the basis $\basis_i$. Suppose that $L$ is $(A_i,B_i)$-compatible in $t_i$ sections with $\basis_i$.

Consider the following computable values:
* $t \in \mathbb{N}$ be the minimal number such that $t_i$ divides $tS_i$ for all $i = 0,\ldots, F-1$,
* $a_i = tS_i/t_i$,
* $A = \text{max}(\lceil\frac{A_i}{S_i}\rceil\ i = 0,\ldots, F-1)$,
* $b_i = AS_i - A_i$,
* $B = \text{min}(B_i\ i = 0,\ldots, F-1)$.

We are going to show that $L$ is $(mA, B)$-compatible in $mt$ sections with $\basis$. Let $n = kmt + r$ where
* $r = r_0m+r_1$ and
* $S_ir_0 + s_i(r_) = r_{2,i}t_i + r_{3,i}$ for $i = 0,\ldots, F-1$.

Then we have that:
$$\frac{L\cdot Q_n(x)}{Q_{n-Am}(x)} = \prod_{i=0}^{F-1}\frac{L\cdot P_{e_i(n)}^{(i)}(x)}{P_{e_i(n)-AS_i}^{(i)}(x)} = 
\prod_{i=0}^{F-1}\frac{L \cdot P_{S_i(kt + r_0) + s_i(r_1)}^{(i)}(x)}{P_{S_i(kt + r_0) + s_i(r_1) - AS_i}^{(i)}(x)} = 
\prod_{i=0}^{F-1}\frac{L \cdot P_{(a_ik + r_{2,i})t_i + r_{3,i}}^{(i)}(x)}{P_{(a_ik + r_{2,i})t_i + r_{3,i} - A_i - b_i}^{(i)}(x)} = 
\prod_{i=0}^{F-1}D_{r_{3,i},A_i+b_i}^{(i)}(a_ik+r_{2,i}),$$
where $D_{*,*}(*)$ are the division polynomials defined in Section 3 with the method ``compatible_division``. Hence we can
apply now the idea of ``equiv_CtD`` to compute the final coefficients $\alpha_{i,j}(k)$ for $\basis$ since we have expressed
the quotient $\frac{L\cdot Q_n}{Q_{n-Am}}$ for each $n = kmt + r$.

### Extending the compatibility of derivations

Let $L$ be a derivation, i.e., $L(PQ) = L(P)Q + PL(Q)$. Then we can compute the compatibility condition in several sections for $\basis$ if $L$ is compatible with all the basis $\basis_i$. Suppose that $L$ is $(A_i,B_i)$-compatible in $t_i$ sections with $\basis_i$.

Consider the following computable values:
* $t \in \mathbb{N}$ be the minimal number such that $t_i$ divides $tS_i$ for all $i = 0,\ldots, F-1$,
* $a_i = tS_i/t_i$,
* $A = \text{max}(\lceil\frac{A_i}{S_i}\rceil\ i = 0,\ldots, F-1)$,
* $b_i = AS_i - A_i$,
* $B = \text{min}(B_i\ i = 0,\ldots, F-1)$.

We are going to show that $L$ is $(mA, B)$-compatible in $mt$ sections with $\basis$. Let $n = kmt + r$ where
* $r = r_0m+r_1$ and
* $S_ir_0 + s_i(r_) = r_{2,i}t_i + r_{3,i}$ for $i = 0,\ldots, F-1$.

Then we have that:
$$\begin{array}{rl}
\displaystyle\frac{L\cdot Q_n(x)}{Q_{n-Am}(x)} = & \displaystyle\sum_{i=0}^{F-1}\left(\frac{L\cdot P_{e_{i}(n)}^{(i)}(x)\displaystyle\prod_{j=0,j\neq i}^{F-1} P_{e_{j}(n)}^{(j)}(x)}{Q_{n-mA}(x)}\right) \\
= & \displaystyle\sum_{i=0}^{F-1} \left(\frac{L\cdot P_{e_i(n)}^{(i)}(x)}{P_{e_i(n)-AS_i}^{(i)}(x)}\displaystyle\prod_{j=0,j\neq i}^{F-1}\frac{P_{e_j(n)}^{(j)}(x)}{P_{e_j(n)-AS_j}^{(j)}(x)}\right)\\
= & \displaystyle\sum_{i=0}^{F-1} \left(\frac{L\cdot P_{(a_ik+r_{2,i})t_i + r_{3,i}}^{(i)}(x)}{P_{(a_ik+r_{2,i})t_i + r_{3,i} - A_i - b_i}^{(i)}(x)}\displaystyle\prod_{j=0,j\neq i}^{F-1}\frac{P_{(a_jk+r_{2,j})t_j + r_{3,j}}^{(j)}(x)}{P_{(a_jk+r_{2,j})t_j + r_{3,j} - A_j - b_j}^{(j)}(x)}\right)\\
= & \displaystyle\sum_{i=0}^{F-1} \left(D_{r_{3,i},A_i+b_i}^{(i)}(a_ik+r_{2,i})\displaystyle\prod_{j=0,j\neq i}^{F-1} I_{(a_jk+r_{2,j})t_j + r_{3,j} - A_j - b_j, A_j+b_j}^{(j)}\right),
\end{array}$$
where $D_{*,*}(*)$ are the division polynomials defined in Section 3 with the method ``compatible_division`` and $I_{*,*}$ are the increasing polynomials defined in Section 3 with the method ``increasing_polynomial``. Hence we can apply now the idea of `equiv_CtD` to compute the final coefficients $\alpha_{i,j}(k)$ for $\basis$ since we have expressed the quotient $\frac{L\cdot Q_n}{Q_{n-Am}}$ for each $n = kmt + r$.

### Example 8

In example 8 of Petkovšek's paper, he looked into a basis built as follows:

* $P_{3k}(n) = \binom{n}{k}\binom{n+k}{2k}$
* $P_{3k+1}(n) = \binom{n}{k}\binom{n+k}{2k+1}$
* $P_{3k+2}(n) = \binom{n}{k+1}\binom{n+k}{2k+1}$

Let $\mathcal{A}_2$ be such basis. This is a clear example of a Shuffled basis. In particular, if we consider $\basis$ the binomial basis and $\basis_2$ the basis we built in Section 7 to represent the second factor, we then have that Petkovšek's basis is exactly the $(1,0,1)$-sieved basis of $(\basis,\basis_2)$.

In [15]:
#def example_8_apery2(n):
#    k = n//3; r = n%3
#    if(r == 0): return binomial(x,k)*binomial(x+k,2*k)
#    elif(r == 1): return binomial(x,k)*binomial(x+k,2*k+1)
#    else: return binomial(x,k+1)*binomial(x+k,2*k+1)
A2 = ShuffledBasis([BinomialBasis,BinomialTypeBasis], [1,0,1])
all(A2[i] == example_8_apery2(i) for i in range(50))

NameError: name 'B2' is not defined

Since both $\basis$ and $\basis_2$ had compatibilities with $X$ and $E$, the basis $\mathcal{A}_2$ has already defined (using an automatic extension) the compatibilities for those two operators:

In [58]:
A2.compatibility_matrix('x')

(
      [      n 2*n + 1]
      [      n   n + 1]
0, 1, [ -n - 1 2*n + 2]
)

In [59]:
A2.compatibility_matrix('E')

(
      [                      1           (4*n - 3/2)/n                     3/2                       1]
      [    (n - 1/2)/(n + 1/2)         1/2*n/(n + 1/2) (3/2*n + 1/2)/(n + 1/2)                       1]
3, 0, [                      0                       1       (3*n + 2)/(n + 1)                       1]
)

And now we can go to the linear difference operator from Example 8:
$$L = (x+2)^2E^2 - (11x^2 + 3x + 25)E - (x+1)^2,$$
and use a similar approach as in the method `DefiniteSumSolutions` to get a recurrence that the sequence $(c_n)_n$ has to satisfy if we assume that $L\cdot \left(\sum_{n\geq 0}c_n \binom{x}{n}\binom{x+n}{2n}\right) = 0$:

In [60]:
L = (x+2)^2*E^2 - (11*x^2 + 33*x + 25)*E - (x+1)^2
recurrence_matrix = A2.recurrence(L)
first_column = [A2.remove_Sni(recurrence_matrix.coefficient((j,0))) for j in range(recurrence_matrix.nrows())]
gcrd = first_column[0].gcrd(*first_column[1:])
gcrd

(n + 1)*Sn - 4*n - 2

Here, the sequence $c_n = \binom{2n}{n}$ is a solution to that recurrence equation. Using the equality
$$\binom{2n}{n}\binom{x}{n}\binom{x+n}{2n} = \binom{x}{n}^2\binom{x+n}{n},$$
we can conclude that the fucntion
$$y(x) = \sum_{n \geq 0} c_n \binom{x}{n}\binom{x+n}{2n} = \sum_{n\geq 0} \binom{x}{n}^2\binom{x+n}{n},$$
is a solution to $L\cdot y = 0$ and, in particular, the sequence $y_k = y(k)$ (which is the Apéry's $\zeta(2)$-sequence) satisfies the recurrence:
$$(k+2)^2 y_{k+2} - (11k^2 + 33^k+25)y_{k+1} - (k+1)^2y_k = 0.$$

##### Example 9

Finally in example 9 of Petkovšek's paper, we consider a new basis with the following format:

* $P_{4n}(x) = \binom{x+n}{2n}^2$
* $P_{4n+1}(x) = \binom{x+n}{2n}\binom{x+n}{2n+1}$
* $P_{4n+2}(x) = \binom{x+n}{2n+1}^2$
* $P_{4n+3}(x) = \binom{x+n}{2n+1}\binom{x+n+1}{2n+2}$

Let $\mathcal{A}_3$ be this basis. In this case, we have a Product basis of the basis $\basis_2$ (`B2`in the code) with itself

In [61]:
def example_9_apery3(n):
    k = n//4; r = n%4
    if(r == 0): return binomial(x+k,2*k)^2
    elif(r == 1): return binomial(x+k,2*k)*binomial(x+k,2*k+1)
    elif(r == 2): return binomial(x+k,2*k+1)^2
    else: return binomial(x+k,2*k+1)*binomial(x+k+1,2*k+2)
A3 = ProductBasis([B2,B2])
all(A3[i] == example_9_apery3(i) for i in range(50))

True

Since $\basis_2$ had compatibilities with $X$ and $E$, the basis $\mathcal{A}_3$ has already defined (using an automatic extension) the compatibilities for those two operators:

In [62]:
A3.compatibility_matrix('x')

(
      [      n 2*n + 1]
      [      n 2*n + 1]
      [ -n - 1 2*n + 2]
0, 1, [ -n - 1 2*n + 2]
)

In [63]:
A3.compatibility_matrix('E')

(
      [                    1           (4*n - 1)/n           (3*n - 1)/n                     2                     1]
      [  (n - 1/2)/(n + 1/2)   (n - 1/2)/(n + 1/2)           n/(n + 1/2) (2*n + 1/2)/(n + 1/2)                     1]
      [                    0                     0                     1                     2                     1]
4, 0, [                    0                     1   (3*n + 5/2)/(n + 1)   (2*n + 3/2)/(n + 1)                     1]
)

And now we can go to the linear difference operator from Example 8:
$$L = (x+2)^3E^2 - (2x+3)(17x^2+51x+39)E + (x+1)^3,$$
and use a similar approach as in the method `DefiniteSumSolutions` to get a recurrence that the sequence $(c_n)_n$ has to satisfy if we assume that $L\cdot \left(\sum_{n\geq 0}c_n \binom{x+n}{2n}^2\right) = 0$:

In [64]:
L = (x+2)^3*E^2 - (2*x+3)*(17*x^2+51*x+39)*E + (x+1)^3
recurrence_matrix = A3.recurrence(L)
first_column = [A3.remove_Sni(recurrence_matrix.coefficient((j,0))) for j in range(recurrence_matrix.nrows())]
gcrd = first_column[0].gcrd(*first_column[1:])
gcrd

(n^2 + 2*n + 1)*Sn - 16*n^2 - 16*n - 4

In this case the sequence $c_n = \binom{2n}{n}^2$ satisfies this recurrence equation. Using the same equality as in example 8, we obtain now that :
$$y(x) = \sum_{n\geq 0} c_n \binom{x+n}{2n}^2 = \sum_{n\geq 0} \binom{x}{n}^2\binom{x+n}{n}^2,$$
is a solution to $L\cdot y = 0$ and, in particular, the sequence $y_k = y(k)$ (which is the Apéry's $\zeta(3)$-sequence) satisfies the recurrence:
$$(k+2)^3y_{k+2} - (2k+3)(17k^2+51k+39)y_{k+1} + (k+1)^3y_{k} = 0.$$

## 8. Some random examples

In Proposition 6, Petkovšek studied the relation of the orders between the associated recurrence equation $\mathcal{R}_{\bbasis}(L)$ and $L$ itself and concluded that the order is exactly the same (so we have the same dimension of solutions and we can map solutions for the operator $L$ to solutions of the operator $\mathcal{R}_\bbasis(L)$.

On the other hand, in the method `DefiniteSumSolutions` and in the two Apery's examples on the previous section, we compute the _greatest common right divisor_ of several operators, which usually returns a recurrence operator of lower order thatn $L$.
We implement here the general method to get that GCRD and use the package to generate random examples and see if we always have at least one solution in the final operator.

In [65]:
def RecurrenceOfSection(basis, operator, sections=1, slice=0):
    recurrence = basis.recurrence(operator, sections=sections)
    if(sections == 1): # nothing to do
        return recurrence
    if(slice < 0 or slice >= sections):
        raise ValueError("The corresponding slice should be between 0 and %d" %(sections-1))
        
    Sni = basis.Sni(); Sn = basis.Sn()
    column = [recurrence.coefficient((j,slice)) for j in range(sections)]
    deg_Sni = [el.degree(Sni) for el in column]
    mdeg = max(deg_Sni)
    column = [basis.OSS()(basis.simplify_operator(el*Sn**mdeg)) for el in column]
    return column[0].gcrd(*column[1:])

In [66]:
examples = []
trivial_examples = 0
while(len(examples) < 50):
    new_element = OE.random_element(degree=randint(0,5))
    if(not new_element in OE.base_ring()):
        sects = [RecurrenceOfSection(B2, new_element, 2, i) for i in range(2)]
        if(sects != [1,1]):
            examples += [(new_element, sects)]
            if(len(examples)%10 == 0):
                print("%d -- %d" %(len(examples), trivial_examples))
        else:
            trivial_examples += 1
            if(trivial_examples%100 == 0):
                print("%d -- %d" %(len(examples), trivial_examples))

2 -- 100
2 -- 200


KeyboardInterrupt: 

In [67]:
[el for el in examples if el[0].order() >= 4]

[]

In [68]:
[el for el in examples if el[0].order() == 3]

[]

In [69]:
[el for el in examples if el[0].order() == 2]

[]

In [70]:
[el for el in examples if el[0].order() == 1]

[((-1/2*x^2 - 3/2*x + 3)*E, [(n^2 + n - 8)*Sn + 4*n^2 - 2*n, 1])]

Here we see that we needed almost 3000 random examples to gather 50 non-trivial examples. From those, we have that only 10 have positive order. I think this is quite normal (since there is no guarantee that the solution space would have a sequence that is the interlacing of two others). However, I still think this is a valid result since this guarantees that there are not those solutions.

In case we want to have all possible solutions we would need to study the whole recurrence matrix.

## 9. Analyzing the whole recursion matrix

After the result from Section 9, we saw that the capability of taking the gcd for the first column not usually provide any result. But that is normal, since we are enforcing that only the first section is not zero. This is very restrictive. however, can we manage to handle other solutions, even if we manage them by sections?

Let's start looking to Example 7.1, where we managed to get a recurrence for the first section:

In [71]:
basis_7_1 = ProductBasis([BinomialBasis(), BinomialBasis()])
M = basis_7_1.recurrence(example_7_1); M

[                 (n + 1)*Sn + (-n - 1) (-3*n)*Sni + (3*n^2 + 3*n + 1)/(n + 1)]
[             (3*n + 3)*Sn + (-3*n - 3)      (n^2 + 2*n + 1)/(n + 2)*Sn + (-n)]

This is a system of linear recurrences. We can write it in such a way that is a first order recurrence system:

In [72]:
basis_7_1.system(example_7_1)

TypeError: bad operand type for unary -: 'list'